In [ ]:
import tensorflow as tf  # Open-source Python library for ML
from tensorflow.python.client import device_lib # Method that enables to list the devices available in the local process 
device_lib.list_local_devices()

In [ ]:
tf.config.experimental.list_physical_devices('CPU')

# Setup

In [ ]:
import numpy as np # *fundamental package for scientific computing in Python. Show documentation
from tensorflow import keras
from tensorflow.keras import layers


# Prepare the data

### 1. Normalization of images: pixels take values in the new range [0,1]
### 2. Define input shape : specify 1-channel images
### 3. One-hot encoding of target values: the representation of categorical variables as binary vectors
![image.png](files/one_hot.png)
Ordinal variables are a good example where a label encoding (integer encoding) would be sufficient.

In categorical variables where no such ordinal relationship exists, the integer encoding is not enough. In fact, using this encoding and allowing the model to assume a natural ordering between categories may result in poor performance.

In [ ]:
# Model / data parameters
num_classes = 10 # *Show image. MNIST 60,000 28x28 grayscale images of the 10 digits, along with a test set of 10,000 images


# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data() # *Show datasets and MNIST information in Keras 
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255 # Pixel normalization rescales the values into a range of [0,1]
x_test = x_test.astype("float32") / 255 # Use a common scale for different features (weights update)

print("x_train shape:", x_train.shape)


# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)# Given a tensor input this operation inserts a dimension of length 1 at the dimension index axis
x_test = np.expand_dims(x_test, -1)# a negative axis counts from the end so axis=-1 adds an inner most dimension #channels


print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

#ADDED to check the format of labels (integers from 0 to 9) before preprocessing (one hot encoding)
print(len(y_train))
print(y_train[59999])

# convert class vectors to binary class matrices             library.function
y_train = keras.utils.to_categorical(y_train, num_classes)  # *Show in Keras. Converts a class vector (integers) to binary class matrix.
y_test = keras.utils.to_categorical(y_test, num_classes) # each numerical/categorical value is represented as a binary vector

#ADDED to check the format of labels (integers from 0 to 9) after preprocessing (one hot encoding)
print(len(y_train))
print(y_train[59999])



# Added for image data visualization

In [ ]:

import matplotlib.pyplot as plt # *Show resources. For creating static, animated, and interactive visualizations in Python
#interactive data visualization
%matplotlib notebook

n=59999 #visualize random image in the training set
plt.imshow(x_train[n,:,:,0], cmap="viridis") #* default colormap 'viridis'. Show cmap="" options


![image.png](files/imgs/cnn.png)

# Build the model

In [ ]:
model = keras.Sequential( #each layer has exactly one input tensor and one output tensor * show in Keras vs. Functional
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, kernel_size=(5, 5), activation="relu"), # * 2D convolution layer. Check arguments, change filters to match picture
        layers.MaxPooling2D(pool_size=(2, 2)), #Downsamples the input along its spatial dimensions. Check arguments
        layers.Conv2D(64, kernel_size=(5, 5), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5), # rate: Float between 0 and 1. Fraction of the input units to drop.
        layers.Dense(num_classes, activation="softmax"),# * recall softmax. *Show images
    ]
)

model.summary() 

#Parameters
# 832= 5x5x32+32
# 51264= 5x5x32x64+64
# 10250= 1024x10+10

## Convolution Operation using a 3x3 kernel:
![image.png](files/imgs/convolution-operation.png)

## Valid padding:
![SegmentLocal](imgs/valid.gif "segment")

## Same padding:
![SegmentLocal](imgs/Same_padding.gif "segment")

## Flattening layer:

![image.png](files/imgs/flattening.png)

# Added for visualization of the architecture (Additional package needs to be installed)

In [ ]:
# NOTE: We need to install pydot> activate the environment and run the next command in Anaconda Prompt: pip install pydot

# `rankdir='LR'` is to make the graph horizontal.Show saved image in default working directory
keras.utils.plot_model(model,show_shapes=True,rankdir='LR',to_file="Sequential_CNN_Model.png") # check the generated image saved in the folder

In [ ]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]) # a function used to judge the performance of model
history= model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2) # Calculates how often predictions equal labels

#Metric functions are similar to loss functions, except that the results from evaluating a metric are not used 
# when training the model but in practice any loss can be used as a metric


## Losses
https://keras.io/api/losses/
## Optimizers
https://keras.io/api/optimizers/
## Metrics
https://keras.io/api/metrics/

## Data splitting
![image.png](files/imgs/MNIST.png)

![image.png](files/imgs/data.png)

## Logits, Softmax, and Loss
<img src="files/imgs/Softmax.png">
Softmax converts logits into probabilities
<img src="files/imgs/cce_loss.png">
Cross-Entropy measures the distance between the output probabilities (P) and the truth values (T)


## Loss: Categorical cross entropy (Classification)
<img src="files/imgs/CategoricalCrossentropy.png">



In [ ]:
#Evaluate the model in the Test set made of 10000 images and compute average loss
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

# Visualize final weights of convolutional kernels

In [ ]:
from matplotlib import pyplot
# retrieve weights from the second hidden layer

filters, biases = model.layers[2].get_weights()

# plot first few filters
n_filters, ix = 8, 1
for i in range(n_filters):
	# get the filter
	f = filters[:, :, :, i]
	# plot each channel separately
	for j in range(8):
		# specify subplot and turn of axis
		ax = pyplot.subplot(n_filters, 8, ix)
		ax.set_xticks([])
		ax.set_yticks([])
		# plot filter channel in grayscale
		pyplot.imshow(f[:, :, j], cmap='gray')
		ix += 1
# show the figure
pyplot.show()

# Visualization of the loss evolution during training

In [ ]:
# summarize history for loss in training and validation sets to assess Overfitting-Underfitting
plt.plot(history.history['loss']) #
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.savefig('Loss vs Epochs.png')

#  Visualization of the classification accuracy during training

In [ ]:
# list all data in history
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()


# Save and load weights 

In [ ]:
#Save the weights of the just trained model in the default directory
model.save_weights('Example1.h5')
model.load_weights('Example1.h5')

# Visualize individual predictions

In [ ]:
index=9999 #index of a randomly picked sample to make a prediction inside the test set (any number from 0 to 9999)
plt.imshow(x_test[index,:,:,0])
y_test_i=y_test[index]
print("\nGround-truth: ",y_test_i)

y_pred_i=model.predict(x_test)[index]
print("\n   Predicted: ",np.around(y_pred_i,decimals=3))


# Computation of the loss for a single sample prediction
### For the batch the only difference is that the average is reported

![title](imgs/CategoricalCrossentropy.png)

In [ ]:
print(y_test_i)
cce = keras.losses.CategoricalCrossentropy()
print("loss(cce) for the single prediction: ",cce(y_test_i, y_pred_i).numpy())

print("\nPredicted: ",y_pred_i)

import math
cce_manual=-math.log(y_pred_i[6])
print(y_pred_i[6])

print(cce_manual)


# Induce underfitting 
### Model is oversimplified to have only a total of 222 parameters

In [ ]:
model2 = keras.Sequential( #each layer has exactly one input tensor and one output tensor * show in Keras vs. Functional
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(1, kernel_size=(5, 5), activation="relu"), # * 2D convolution layer. Check arguments, change filters to match picture
        layers.MaxPooling2D(pool_size=(2, 2)), #Downsamples the input along its spatial dimensions. Check arguments
        layers.Conv2D(1, kernel_size=(5, 5), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax"),# * recall softmax. *Show images
    ]
)

model2.summary() 

In [ ]:
batch_size = 128
epochs = 30
# Compile model
model2.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]) # a function used to judge the performance of model
# Fit the model
history2 = model2.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2) # Calculates how often predictions equal labels


In [ ]:


# summarize history for loss in training and validation sets to assess Overfitting-Underfitting
plt.plot(history.history['loss']) #
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')

plt.plot(history2.history['loss']) #
plt.plot(history2.history['val_loss'])
plt.legend(['train_original', 'val_original','train_new', 'val_new'], loc='upper left')
plt.ylim(0,1)
plt.show()
#

In [ ]:
index=0 #index of a randomly picked sample to make a prediction inside the test set
plt.imshow(x_test[index,:,:,0])
y_test_i=y_test[index]
print("\nGround-truth: ",y_test_i)

y_pred_i=model2.predict(x_test)[index]
print("\nPredicted: ",np.around(y_pred_i,decimals=3))


# Induce overfitting
## Training data will be decreased to 60 samples only (by setting split to 0.999)

In [ ]:
model3 = keras.Sequential( #each layer has exactly one input tensor and one output tensor * show in Keras vs. Functional
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, kernel_size=(5, 5), activation="relu"), # * 2D convolution layer. Check arguments, change filters to match picture
        layers.MaxPooling2D(pool_size=(2, 2)), #Downsamples the input along its spatial dimensions. Check arguments
        layers.Conv2D(64, kernel_size=(5, 5), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax"),# * recall softmax. *Show images
    ]
)

model3.summary()

In [ ]:
batch_size = 128
epochs = 30
# Compile model
model3.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]) # a function used to judge the performance of model
# Fit the model
history3 = model3.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.999) # Calculates how often predictions equal labels


In [ ]:
import matplotlib.pyplot as plt # *Show resources. For creating static, animated, and interactive visualizations in Python
#interactive data visualization
%matplotlib notebook 

# summarize history for loss in training and validation sets to assess Overfitting-Underfitting
plt.plot(history.history['loss']) #
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')

plt.plot(history3.history['loss']) #
plt.plot(history3.history['val_loss'])
plt.legend(['train_original', 'test_original','train_new', 'test_new'], loc='upper left')
plt.show()



# Run previous example as a Regression problem
### 1. Do not one-hot encode the labels, instead use the original numeric integer labels
### 2. Substitute output layer of neurons(num_classes) to one single output neuron and change the activation from softmax to linear
### 3. Use a different Loss: Mean Squared Error (Regression)
<img src="files/imgs/MSE.png">

In [ ]:


# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data() # *Show datasets and MNIST information in Keras 
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255 # Pixel normalization rescales the values into a range of [0,1]
x_test = x_test.astype("float32") / 255 # Use a common scale for different features (weights update)

print("x_train shape:", x_train.shape)


# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)# Given a tensor input this operation inserts a dimension of length 1 at the dimension index axis
x_test = np.expand_dims(x_test, -1)# a negative axis counts from the end so axis=-1 adds an inner most dimension #channels


print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

#ADDED to check the format of labels (integers from 0 to 9) 
print(len(y_train))
print(y_train[59999])




# Build a model with single output neuron and linear activation

In [ ]:
model4 = keras.Sequential( #each layer has exactly one input tensor and one output tensor * show in Keras vs. Functional
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, kernel_size=(5, 5), activation="relu"), # * 2D convolution layer. Check arguments, change filters to match picture
        layers.MaxPooling2D(pool_size=(2, 2)), #Downsamples the input along its spatial dimensions. Check arguments
        layers.Conv2D(64, kernel_size=(5, 5), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(1, activation="linear"),
    ]
)

model4.summary() 

In [ ]:
batch_size = 128
epochs = 15

opt=tf.keras.optimizers.Adam(
    learning_rate=0.005,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam"
)

model4.compile(loss="mean_squared_error", optimizer=opt) # a function used to judge the performance of model
history4= model4.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2) # Calculates how often predictions equal labels

#Metric functions are similar to loss functions, except that the results from evaluating a metric are not used 
# when training the model but in practice any loss can be used as a metric

In [ ]:
#Evaluate the model in the Test set made of 10000 images and compute average loss
score = model4.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score)

In [ ]:
index=0 #index of a randomly picked sample to make a predcition inside the test set
plt.imshow(x_test[index,:,:,0])
y_test_i=y_test[index]
print("\nGround-truth: ",y_test_i)

y_pred_i=model4.predict(x_test)[index]
print("\nPredicted: ",np.around(y_pred_i,decimals=3))


# Next hands-on session: Transfer learning & fine-tuning 
## "Taking features learned on one problem, and leveraging them on a new, similar problem."

##  An end-to-end example: fine-tuning an image classification model

## Keras provides deep learning models that are made available alongside pre-trained weights
https://keras.io/api/applications/

## ResNet
https://arxiv.org/pdf/1512.03385.pdf
<img src="files/imgs/ResNet.png">


In [ ]:
import tensorflow as tf

# Use a base model or backbone for feature extraction
base_model=tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(112, 112, 3)
    )

# Create new model on top.
inputs = keras.Input(shape=(112,112,3))
x = base_model(inputs)
#Add additional layers on top
x = keras.layers.Flatten()(x)
outputs = keras.layers.Dense(1, activation="linear")(x)
model5 = keras.Model(inputs, outputs)
print(model5.summary())

# let's visualize layer names and layer indices
for i, layer in enumerate(model5.layers):
    print(i, layer.name)
      


In [ ]:
keras.utils.plot_model(model5,show_shapes=True,rankdir='LR',to_file="Transfer_learning_Model.png") 